# Team Project 3
Members:

Neal Fennimore - nf2137

Nathan Vulakh - nv2155

Mike Dela Calzada - md5286

Miri Alair - mg6931

# Problem 1 

> Describe one time pad and explain the reason it meets perfect security.

A One-Time Pad is an unbreakable encryption system if certain conditions are met. The algorithm is an improvement of the Vernam cipher where plain text is encrypted using its own key. There are a couple of requirements for the One-Time pad to ensure perfect secrecy. The first requirement is the use of a truly random generated key that is as long as the size of the plain text message. The second requirement is that the key used to encrypt/decrypt is used only once, and then discarded.

In a One-Time pad, a number is assigned to each character of the plain text. The number of possible encryptions increases exponentially with a large message. For example, to encrypt ‘ALICE’ the number of possibilities is almost 12 million (26^5 --> 11,881,376). This can also be a problem because as the One-Time pad makes the large quantities of random keys, there is a need to handle the immense key distribution.


# Problem 2 

> Write a python programming code (or your favorite programing language) to realize the onetime Pad’s encryption and decryption algorithms. It should mirror the communication between Alice and Bob.

Alice starts one program that asks Alice to enter a message on screen. The program then outputs the
ciphertext on screen. It saves the cyphertext in hex in one file and the key in hex in another file. We
assume that the ciphertext file is publicly available, which mimics that public communication. We also
assume that Bob has a secret way to get the key file, which mimics the key distribution. In our case, we
can simply put them into two different folders.

Bob starts another program that reads the key and ciphertext and outputs the plaintext on screen.

In short, what we do here is to separate the encryption and decryption process into two programs
(applications).


##### Alice's program

In [11]:
import os
import hashlib

def encrypt(plaintext: str, key: bytes) -> bytes:
    return bytes([p ^ k for p, k in zip(plaintext.encode(), key)])

def generate_random_key(length: int) -> bytes:
    return os.urandom(length)

def compute_sha256(message: str) -> str:
    return hashlib.sha256(message.encode()).hexdigest()

def input_write():
    if not os.path.exists("ciphertext_folder"):
        os.makedirs("ciphertext_folder")

    if not os.path.exists("key_folder"):
        os.makedirs("key_folder")

    message = input("Message: ")
    message_hash = compute_sha256(message)
    combined_message = message + message_hash
    key = generate_random_key(len(combined_message))

    ciphertext = encrypt(combined_message, key)
    print(f"Ciphertext: {ciphertext.hex()}")

    with open("ciphertext_folder/ciphertext.txt", "wb") as f:
        f.write(ciphertext)

    with open("key_folder/key.txt", "wb") as f:
        f.write(key)

if __name__ == "__main__":
    input_write()





Ciphertext: e8f80ec0e07737e431d7f381d62f0aea3eb781c94b12a36da2077aa2aa9a3e5b7fe06671527680170edfacedb5360892fa7c4aa6b9cbae4dbd258b188186480ed92082


##### Bob's program

In [ ]:
def decrypt(ciphertext: bytes, key: bytes) -> str:
    return bytes([c ^ k for c, k in zip(ciphertext, key)]).decode('utf-8')

def compute_sha256(message: str) -> str:
    return hashlib.sha256(message.encode()).hexdigest()

def get_from_input() -> bytes:
    while True:
        try:
            hex_string = input("Please enter the ciphertext (hex): ")
            return bytes.fromhex(hex_string)
        except ValueError:
            print("Invalid hex. Please try again.")

def get_from_file(file: str):
    with open(file, "rb") as f:
        key = f.read()
    return key

def choices():
    global ciphertext
    while True:
        choice = input("Input the ciphertext or read from file? (input/file): ").strip().lower()

        if choice == 'input':
            ciphertext = get_from_input()
            break
        elif choice == 'file':
            filename = input("What is the file that you want to read the ciphertext from? Include file extension: ")
            filepath = os.path.join("ciphertext_folder", filename)
            if os.path.exists(filepath):
                ciphertext = get_from_file(filepath)
                break
            else:
                print("I can't find that file, try again.")
                continue
        else:
            print("Please type either input or file. Try again.")

    key = get_from_file("key_folder/key.txt")

    decrypted_message = decrypt(ciphertext, key)

    message_hash = decrypted_message[-64:]
    decrypted_message = decrypted_message[:-64]

    if compute_sha256(decrypted_message) != message_hash:
        print("Incorrect ciphertext, exiting program.")
        return

    print(f"Plaintext: {decrypted_message}")

if __name__ == "__main__":
    choices()


# Problem 3 

> Write a python programming code (or your favorite programing language) to realize manytime pad. This code is an extension to the code written for the one-time pad. It hardcodes 10 plaintext
messages you have chosen as a list in the code and uses one randomly generated key to encrypt (xor) all
the plaintext messages. The code then saves the plaintext in hex first, then the key in hex and the
ciphertexts in hex into one file. Assume the key length is longer than any of the plaintexts.

The purpose of this problem is to see if there are any recognizable patterns by observing the outputs.
You can change the plaintexts or the key to verify your findings. These findings would be useful in the
next problem.


##### Program to write ciphertexts and many time pad key to file

In [10]:
def get_key_from_file(filename): 
    with open(filename + ".txt", "r") as f:
        key = bytes.fromhex(f.readlines()[0].strip())
    return key

def get_numbered_ciphertext_from_file(filename, ciphertext_number):
    with open(filename + ".txt", "r") as f:
        ciphertext = bytes.fromhex(f.readlines()[ciphertext_number].strip())
    return ciphertext

def get_key_and_write_to_file(filename):
    key =  os.urandom(4096) # way too long key to ensure we can encrypt the messages
    with open(filename + ".txt", "w") as f:
        f.write(key.hex() + "\n")
    return key

def get_and_encrypt_input():
    filename = input("File name for messages (no extension)")
    key = get_key_and_write_to_file(filename)
    ciphertexts = []
    with open(filename + ".txt", "a") as f:
        for x in range(1, 11):
            message = input("Message (" + str(x) + "): ")
            message_hash = compute_sha256(message)
            combined_message = message + message_hash
            ciphertext = encrypt(combined_message, key)
            #print(f"Ciphertext: {ciphertext.hex()}")
            f.write(ciphertext.hex() + "\n")

if __name__ == "__main__":
    get_and_encrypt_input()


##### Program to read and display specific ciphertexts from list

In [ ]:
def display_plaintexts():
    filename = input("File name where messages are stored (no extension)")         
    key = get_key_from_file(filename)

    with open(filename + ".txt", "r") as f:
        lines = f.readlines()
        for x in range(1, 11):
            ciphertext = bytes.fromhex(lines[x].strip())
            decrypted_msg = decrypt(ciphertext, key)

            message_hash = decrypted_msg[-64:]
            decrypted_message = decrypted_msg[:-64]

            if compute_sha256(decrypted_message) != message_hash:
                print("Incorrect ciphertext, exiting program.")
                return

            print(decrypted_message)
            
if __name__ == "__main__":
    display_plaintexts()



this
is
nathan
I
am
testing
this
program
ok
bye


# Problem 4

> Let us make an attack on the many-time pad. Assume Eva has collected the 10 ciphertexts
and she knew they are generated by the same key. In addition, we assume all the plaintexts are in
English. Space, comma, period, and question mark are being used in the plaintext, but no other special
characters are allowed. Eva wants to use the information to decrypt the last message (target message).
Our job is to verify if this is possible, at least partially to get the target plaintext. You need to design your
attacking strategy, convert it into a pseudo-code and realize it in python code or any your programming
language you chose. Remember, cryptoanalysis needs a human in the loop and a bit of luck. The hints
below may give you some ideas. Your submission should include the description of your strategy, the
pseudo code, the actual code and the target plaintext that is decrypted completely or partially. For
partial cracking, students need to provide insights why they could not go further.

Collected ciphertexts:

1. 71fe1ace4389087266117cd7c98c4182851b3acff3b086e3f83f94d6eb05c4ba85d8e1fa14f11d1c3
b568ff6cff5c09c5d67ef5c9c71b7eeb3d45a5154ab17b83e071ce9d8988adb4afedf46a840

2. 71fe1ace559a1e7266117cd7ce8745d7be2e74c3f0f68eeef57e8884e607debf81dfa0f012f958196
81ae7f29fe4839b5175ef5e8760bef0b9d44b504eba12b22f5404f89dd085d550a48865a14f9b15a
94dabe609ca2df2cccf210cefdb1af5389719795e1f0179cb77c5c456954d88f3

3. 72fe069c51c81a20775928c7879d4fd2a93c3acff3f69fe5fe2e9493a303d9ea98c4e5b60ae40a146
058e7c787fbd09a1474e25dc865b5e6af865d4a40a61bfd384e06e0cfc1ccd356ff8853ac438905fa
5fe3fd41cb3bbc8ac9

4. 67e543885b9a5b2267177084cf8453ccb8633ad7fdb39de5b13f8a93a304d6bf8bc4f4ef5def110b
6f56a3e186e2c68c1470ef5c9c2ffbd6a291571e40ba1afd3b4b1fe0c4cbccc15df5dc07b043da01fa
6ae4fd158f37b3c0cd

5. 71fe029a148c1236320d7192878a59cfbc3a6ec5e7f68befb13196d6ea1ec4ea81d9e3fe50ea0f19
6d02a2f7cfe2c29c5577e35d8630baf6ea80465b01aa1abc394f57a1f4ccccda59ff8846e44b8805b
b5cabe608c231f2dec8364ae7d90ab4358c5c3a421b06

6. 6ef914ce5989152b321a769ad79c42c7be6f6ad2fab19de1fc339d84f04ad3a589dfa0ff09ab0c196f
13e7e780b4c097556ded57c871fbeea393464a01aa0ab1381848cfd2d6898918efc046b00b8940b
b08e3f313cb23b3dfd8645cfcd80ff82489

7. 71fe1ace4389087266117cd7c4865bd2b93b7fd2b5a58ce9f4308c9ff01e97ab82cbf2ef5dfc101d6
a56b3fb8ab4d08b4167ef5c9c30b8f0ab97455b45e81efd364605e49ddb83df48eedc42b60c900fb
14db4b229ca74b6c4d96442e1c34df8288f5c3a450a527ecc7c82865b8e

8. 71fe029a148c1437615978d7c58854dbec2c75cde5a39be5e37e9b97ef0697a285dfa0f01cff101d
764983f29bf5

9. 71fe1ace50875b31730d6ad7cb8640c7ec3c73d4e1bf81e7b13796d6e518d8a4988ceff05dff101d
2415a8fe9fe1d79a4623eb5e8430bfe3b3d442514faf40fd18420be0c8cb89924cf3cd5ee448950ef
d5cabe500c120f2d9d26440ebc34de029811977430b01748276d79012955cc6a65aebb9054becd
a5c9278

10. 71fe029a1483123c76597691878459cca9363ac4faf68ceffc2e8d82e61897b98fc5e5f809e20b0c7
756b2e08aab83bc5560e257

Target Message:

71fe0680149d083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd898e61cd2b8ccd9f3f35dff101d241d
a2eacff9cc8d5123fe5a897efbeda4974b


Here are some observations that might be helpful to your attack thinking.


1. 0 Ꚛ x = x for any x in {0, 1}
2. 1 Ꚛ x = 1-x (in other words, it is the opposite)
3. kꚚk = 0 for any bit string.
4. c = m Ꚛ k
5. m = c Ꚛ k
6. c Ꚛ m = k
7. Any two ciphertext c1 = m1 Ꚛ k, c2 = m2 Ꚛ k, we can do c1 Ꚛ c2 = m1 Ꚛ m2. That is, we
know the result of m1 Ꚛ m2. This shall tell us something, especially when spaces or
other no letters or numbers in the plaintexts.
You can also get more information on m1 by doing c1 Ꚛ ci, i = 2, 3, …, 10, and use them
to confirm your findings. You can do similar operations to the other message, mi as well.
8. The key patterns can be obtained from m and c using k = c Ꚛ m. That is, if we know
something about m, then we know such thing will be manifested in k. Moreover, we
have 10 messages from which we shall know a lot about the key.
9. Here comes one interesting pattern. From ASCII table, the letter set A-Z are A = 0x41 =
0100 0001, … Z = 0x5A = 0101 1010, The first three bits together gives a pattern 010.
The letter set a-z are a = 0x61 = 0110 1001, … z = 0x7A = 0111 1010. The first three bits
together gives 011. We know the space = 0x20 = 0010 0000. So we can see that space Ꚛ
with a capital letter = a little letter and space Ꚛ with a little letter = a capital letter. But
any two letters XOR will give no letters as the first two bits is 00. This observation can be
applied in mi Ꚛ mj.
10. Students can do more pattern findings. For example, for any number in ASCII, it always
starts 0011.
11. Other than the ASCII patterns, students may try other methods such as known plaintext
attack called “crib dragging” using common words like “The”, “I”, etc. repeated words,
frequency analysis, short segment dictionary attack, etc.
12. Once we have the key structure, even partially, we can decipher the target ciphertext.
More often, we need to fill the gap by guessing and verification. 




In [20]:
from itertools import combinations, product
from string import ascii_letters
from sys import byteorder
from collections import Counter
from typing import List, Optional

keyspace = set(ascii_letters + ' ')

known_ciphertexts: List[bytearray] = list(map(lambda x: bytearray(bytes.fromhex(x)), [
    "71fe1ace4389087266117cd7c98c4182851b3acff3b086e3f83f94d6eb05c4ba85d8e1fa14f11d1c3b568ff6cff5c09c5d67ef5c9c71b7eeb3d45a5154ab17b83e071ce9d8988adb4afedf46a840",
    "71fe1ace559a1e7266117cd7ce8745d7be2e74c3f0f68eeef57e8884e607debf81dfa0f012f95819681ae7f29fe4839b5175ef5e8760bef0b9d44b504eba12b22f5404f89dd085d550a48865a14f9b15a94dabe609ca2df2cccf210cefdb1af5389719795e1f0179cb77c5c456954d88f3",
    "72fe069c51c81a20775928c7879d4fd2a93c3acff3f69fe5fe2e9493a303d9ea98c4e5b60ae40a146058e7c787fbd09a1474e25dc865b5e6af865d4a40a61bfd384e06e0cfc1ccd356ff8853ac438905fa5fe3fd41cb3bbc8ac9",
    "67e543885b9a5b2267177084cf8453ccb8633ad7fdb39de5b13f8a93a304d6bf8bc4f4ef5def110b6f56a3e186e2c68c1470ef5c9c2ffbd6a291571e40ba1afd3b4b1fe0c4cbccc15df5dc07b043da01fa6ae4fd158f37b3c0cd",
    "71fe029a148c1236320d7192878a59cfbc3a6ec5e7f68befb13196d6ea1ec4ea81d9e3fe50ea0f196d02a2f7cfe2c29c5577e35d8630baf6ea80465b01aa1abc394f57a1f4ccccda59ff8846e44b8805bb5cabe608c231f2dec8364ae7d90ab4358c5c3a421b06",
    "6ef914ce5989152b321a769ad79c42c7be6f6ad2fab19de1fc339d84f04ad3a589dfa0ff09ab0c196f13e7e780b4c097556ded57c871fbeea393464a01aa0ab1381848cfd2d6898918efc046b00b8940bb08e3f313cb23b3dfd8645cfcd80ff82489",
    "71fe1ace4389087266117cd7c4865bd2b93b7fd2b5a58ce9f4308c9ff01e97ab82cbf2ef5dfc101d6a56b3fb8ab4d08b4167ef5c9c30b8f0ab97455b45e81efd364605e49ddb83df48eedc42b60c900fb14db4b229ca74b6c4d96442e1c34df8288f5c3a450a527ecc7c82865b8e",
    "71fe029a148c1437615978d7c58854dbec2c75cde5a39be5e37e9b97ef0697a285dfa0f01cff101d764983f29bf5",
    "71fe1ace50875b31730d6ad7cb8640c7ec3c73d4e1bf81e7b13796d6e518d8a4988ceff05dff101d2415a8fe9fe1d79a4623eb5e8430bfe3b3d442514faf40fd18420be0c8cb89924cf3cd5ee448950efd5cabe500c120f2d9d26440ebc34de029811977430b01748276d79012955cc6a65aebb9054becda5c9278",
    "71fe029a1483123c76597691878459cca9363ac4faf68ceffc2e8d82e61897b98fc5e5f809e20b0c7756b2e08aab83bc5560e257",
]))
target_ciphertext = bytearray(bytes.fromhex("71fe0680149d083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd898e61cd2b8ccd9f3f35dff101d241da2eacff9cc8d5123fe5a897efbeda4974b"))

def xor(x: bytes, y: bytes) -> bytearray:
    return bytearray(a ^ b for a, b in zip(x, y))

# mismatch = 0
# def decode(c1: bytearray, c2: bytearray, c3: bytearray, key: bytearray):
#     global mismatch
#     assert(len(c1) == len(c2) and len(c2) == len(c3))

#     for i in range(len(c1)):
#         x1 = c1[i] ^ c2[i]
#         x2 = c1[i] ^ c3[i]

#         # c1[i] is most likely a space
#         if x1 >= 65 and x2 >= 65:
#             k = c1[i] ^ ord(' ')

#             m1 = chr(c1[i] ^ k)
#             m2 = chr(c2[i] ^ k)
#             m3 = chr(c3[i] ^ k)

#             assert(m1 == ' ')

#             if m2 not in keyspace or m3 not in keyspace:
#                 continue

#             if key[i] != ord('-'):
#                 mismatch += 1
#                 print('Key mismatch at index', i)
#             key[i] = k

def track_space(result: bytearray) -> Counter:
    space_ctr = Counter()

    for idx, c in enumerate(result):
        if c == 0 or chr(c) in keyspace:
            space_ctr[idx] += 1

    return space_ctr


def recover_key_portion(ciphertexts: List[bytearray]) -> List[Optional[int]]:
    key: List[Optional[int]] = [
        None for _ in range(min(map(len, ciphertexts)))
    ]

    for c1_idx, c1 in enumerate(ciphertexts):
        ctr = Counter()

        for c2_idx, c2 in enumerate(ciphertexts):

            if c1_idx == c2_idx:
                continue

            tracker = track_space(xor(c1, c2))
            ctr.update(tracker)

        for idx, count in ctr.items():
            if count == len(ciphertexts) - 1:
                key[idx] = c1[idx] ^ ord(' ')

    return key


sorted_known_ciphertexts = sorted(known_ciphertexts, key=len)
key: List[Optional[int]] = []

while len(sorted_known_ciphertexts) > 1:
    key += recover_key_portion(sorted_known_ciphertexts)

    length = len(sorted_known_ciphertexts[0])
    sorted_known_ciphertexts = [
        c[length:]
        for c in sorted_known_ciphertexts[1:]
    ]

replacement_key = '?'
key_with_spaces_found = bytearray([ x if x is not None else ord(replacement_key) for x in key ])

for c in known_ciphertexts:
    print(xor(c, key_with_spaces_found).decode(errors='ignore'))


print("\nTarget:")
print(xor(target_ciphertext, key_with_spaces_found).decode(errors='ignore'))


Ny was thCn~ I$ ffia hosptal+ze#iH cidctNly eouc(e8#he frwal
Ny are thCizurne d remius f-r &W% p evao_es tnor-ok;y hih Bepauethey fe wa ca sFiHg drnu
Mere are  ppe f ope in he 5or+_g hs wb Znerbtan$ q9ary n th|sewo don 
X for punOhlnt\ he ae nauhtybdi4Pidie sct heh ar% t ays et t| aBot caj
Nat did tN fmptr  o its uchoaw&R=e aatbna tye b%aph It a a trettime trfg 
e }$t
Qw many cIp}erPpogmmrs dos i6 t&P, ocanh N igyt b5l'wNone hat4s  ardwau pbl
Ny was thCcdpue senist agrybwh"Uitesudctcaczed ! 	y:e copter3joe He dn n le z5 AnC bi
Nat does Gbky opur all hs f#th"IvDt
Ny do catUle iti i fron ofbth"*optr aldy }ong '}4ausetey wontwant s l t m|4sK Iut c  
Nat kind I fne	 o mpter sien6is3Hiue ach

Target:
Nen using }remcierneverusebth""e oe eaA nct


In [78]:
hex_1_clear = str.encode('why was the new IT official hospitalized? He accidentally touched the firewall')
hex_1 = bytes.fromhex('71fe1ace4389087266117cd7c98c4182851b3acff3b086e3f83f94d6eb05c4ba85d8e1fa14f11d1c3b568ff6cff5c09c5d67ef5c9c71b7eeb3d45a5154ab17b83e071ce9d8988adb4afedf46a840')

hex_2_clear = str.encode('why are the insurance and premiums for all app developers enormously high? Because they are always')
hex_2 = bytes.fromhex('71fe1ace559a1e7266117cd7ce8745d7be2e74c3f0f68eeef57e8884e607debf81dfa0f012f95819681ae7f29fe4839b5175ef5e8760bef0b9d44b504eba12b22f5404f89dd085d550a48865a14f9b15a94dabe609ca2df2cccf210cefdb1af5389719795e1f0179cb77c5c456954d88f3')

hex_3_clear = str.encode('there are 10 types of people in the world. Those who understand binary and those who don\'t')
hex_3 = bytes.fromhex('72fe069c51c81a20775928c7879d4fd2a93c3acff3f69fe5fe2e9493a303d9ea98c4e5b60ae40a146058e7c787fbd09a1474e25dc865b5e6af865d4a40a61bfd384e06e0cfc1ccd356ff8853ac438905fa5fe3fd41cb3bbc8ac9')

hex_5_clear = str.encode('what did the computer do on its much-awaited vacation at the beach? It had a great time surfing the net')
hex_5 = bytes.fromhex('71fe029a148c1236320d7192878a59cfbc3a6ec5e7f68befb13196d6ea1ec4ea81d9e3fe50ea0f196d02a2f7cfe2c29c5577e35d8630baf6ea80465b01aa1abc394f57a1f4ccccda59ff8846e44b8805bb5cabe608c231f2dec8364ae7d90ab4358c5c3a421b06')

hex_6_clear = str.encode('how many computer programmers does it take to change a light bulb? None; that\'s a hardware problem')
hex_6 = bytes.fromhex('6ef914ce5989152b321a769ad79c42c7be6f6ad2fab19de1fc339d84f04ad3a589dfa0ff09ab0c196f13e7e780b4c097556ded57c871fbeea393464a01aa0ab1381848cfd2d6898918efc046b00b8940bb08e3f313cb23b3dfd8645cfcd80ff82489')

hex_7_clear = str.encode('why was the computer scientist angry when the student cracked a lame computer joke? He did not like')
hex_7 = bytes.fromhex('71fe1ace4389087266117cd7c4865bd2b93b7fd2b5a58ce9f4308c9ff01e97ab82cbf2ef5dfc101d6a56b3fb8ab4d08b4167ef5c9c30b8f0ab97455b45e81efd364605e49ddb83df48eedc42b60c900fb14db4b229ca74b6c4d96442e1c34df8288f5c3a450a527ecc7c82865b8e')

hex_9_clear = str.encode('why do cats love sitting in front of the computer all day long? Because they don\'t want to let the mouse')
hex_9 = bytes.fromhex('71fe1ace50875b31730d6ad7cb8640c7ec3c73d4e1bf81e7b13796d6e518d8a4988ceff05dff101d2415a8fe9fe1d79a4623eb5e8430bfe3b3d442514faf40fd18420be0c8cb89924cf3cd5ee448950efd5cabe500c120f2d9d26440ebc34de029811977430b01748276d79012955cc6a65aebb9054becda5c9278')
# print('hex 1 from 7', xor(hex_1, xor(hex_7, hex_7_clear)).decode(errors='ignore'))
# print('hex 1 from 3', xor(hex_1, xor(hex_3, hex_3_clear)).decode(errors='ignore'))
# print('hex 2 from 7',xor(hex_2, xor(hex_7, hex_7_clear)).decode(errors='ignore'))
# print('hex 3 from 7',xor(hex_3, xor(hex_7, hex_7_clear)).decode(errors='ignore'))
print('hex 5 from 7',xor(hex_5, xor(hex_7, hex_7_clear)).decode(errors='ignore'))
print('hex 5 from 9',xor(hex_5, xor(hex_9, hex_9_clear)).decode(errors='ignore'))
# print('hex 7 from 1',xor(hex_7, xor(hex_1, hex_1_clear)).decode(errors='ignore'))
# print('hex 7 from 3',xor(hex_7, xor(hex_3, hex_3_clear)).decode(errors='ignore'))
# print('hex 7 from 2',xor(hex_7, xor(hex_2, hex_2_clear)).decode(errors='ignore'))
print('hex 7 from 5',xor(hex_7, xor(hex_5, hex_5_clear)).decode(errors='ignore'))

print('hex 7 from 6',xor(hex_7, xor(hex_6, hex_6_clear)).decode(errors='ignore'))
print('hex 9 from 7',xor(hex_9, xor(hex_7, hex_7_clear)).decode(errors='ignore'))
print('hex 9 from 5',xor(hex_9, xor(hex_5, hex_5_clear)).decode(errors='ignore'))

target = bytes.fromhex('71fe0680149d083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd898e61cd2b8ccd9f3f35dff101d241da2eacff9cc8d5123fe5a897efbeda4974b')
print(len(hex_9_clear), len(target))
print('\n\ntarget from 6:',xor(target, xor(hex_6, hex_6_clear)).decode(errors='ignore'))
print('target from 9:',xor(target, xor(hex_9, hex_9_clear)).decode(errors='ignore'))



hex 5 from 7 what did the computer do on its much-awaited vacation at the beach? It had a great time surfing the
hex 5 from 9 what did the computer do on its much-awaited vacation at the beach? It had a great time surfing the net
hex 7 from 5 why was the computer scientist angry when the student cracked a lame computer joke? He did not like it 
hex 7 from 6 why was the computer scientist angry when the student cracked a lame computer joke? He did not lik
hex 9 from 7 why do cats love sitting in front of the computer all day long? Because they don't want to let the 
hex 9 from 5 why do cats love sitting in front of the computer all day long? Because they don't want to let the mous
104 59


target from 6: when using a stream cipher never use the key more than once
target from 9: when using a stream cipher never use the key more than once


#### Plaintext for the target in Problem 4: `when using a stream cipher never use the key more than once`

# Problem 5 (optional) 

> (optional) How many ciphertexts in many-time pad should we collect before we can do
meaningful attack? This is an open-ended discussion. 
